In [ ]:
import pandas as pd
import re
import csv
from ast import literal_eval
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **俄罗斯数据集(Российский набор данных)**

In [ ]:
# 读取wildberries数据集（Ознакомьтесь с набором данных wildberries）
csv_wildberries = "/content/wildberries冬季外套数据.csv"
csv_data_wildberries = pd.read_csv(csv_wildberries, low_memory = False)
csv_df_wildberries = pd.DataFrame(csv_data_wildberries)
csv_df_wildberries

,title,brand,price,count,rating,info_dict
0,Пальто демисезонное экомех оверсайз тедди,Baimuni,7087,665.0,4.7,"{'Состав': 'Эко кожа 50%; Эко шерсть 50%', 'Цв..."
1,Куртка весна с капюшоном,TANIR,4597,126.0,4.7,"{'Состав': 'полиэстер', 'Цвет': 'черный', 'Сез..."
2,Косуха куртка серая оверсайз,Moraru,2316,17929.0,4.6,"{'Состав': 'экокожа 100%', 'Цвет': 'бело-серый..."
3,Пальто демисезонное шерстяное оверсайз,OVERCOAT,12203,1081.0,4.4,"{'Состав': 'шерсть 90 %; вискоза 10 %', 'Цвет'..."
4,Пальто женское оверсайз весна-осень демисезонное,BREGATO,4722,492.0,4.6,"{'Состав': 'вискоза; шерсть', 'Цвет': 'черный'..."
...,...,...,...,...,...,...
4572,Куртка,ФРАНТишка,2625,3.0,3.7,"{'Состав': 'полиэстер 100%', 'Цвет': 'кораллов..."
4573,Комплект зима куртка брюки костюм теплый,JEMI,2762,NaN,Нет оценок,"{'Состав': 'полиэстер 41%; нейлон 59%', 'Цвет'..."
4574,Зимняя короткая куртка оверсайз,LUIZA,8168,21.0,4.9,"{'Состав': 'полиэстер 100%', 'Цвет': 'светлый ..."
4575,пальто пуховик,GRANDSHOP,4291,NaN,Нет оценок,"{'Состав': 'Полиэстер 100', 'Цвет': 'бежевый л..."


数据清洗（очистка данных）

In [ ]:
# 存储处理后的数据（Хранить обработанные данные）
wildberries_data_1 = []

# 打开名为csv_wildberries的文件，以只读模式打开，使用utf-8编码（Откройте файл с именем csv_wildberries, откройте его в режиме только для чтения, используйте кодировку utf-8）
with open(csv_wildberries, 'r', encoding = 'utf-8') as file:
    # 创建一个CSV字典阅读器对象（Создайте объект для чтения словаря CSV）
    reader = csv.DictReader(file)
    # 遍历读取每一行数据，将其作为字典添加到列表wildberries_data_1中（Просмотрите и прочитайте каждую строку данных и добавьте ее в качестве словаря в список wildberries_data_1）
    for row in reader:
        wildberries_data_1.append(row)


# 删除重复值（Удаление повторяющихся значений）
csv_df_wildberries.drop_duplicates(inplace=True)

# 将数据集中的大写字母转换为小写字母（Преобразуйте заглавные буквы в наборе данных в строчные）
csv_df_wildberries = csv_df_wildberries.apply(lambda x: x.astype(str).str.lower())

Список “name”

In [ ]:
# 定义一个函数来处理“title"列中的字符串(Определите функцию для обработки строки в столбце "заголовок")
def process_title_r(title):
    # 去除符号(Удалить символы)
    cleaned_title = re.sub(r'[^\w\s]', '', title)
    # 将俄语大写字母变为小写字母(Замените русские заглавные буквы на строчные)
    cleaned_title = cleaned_title.lower()
    return cleaned_title

# 应用函数到title列(Примените функцию к столбцу заголовка)
csv_df_wildberries['title'] = csv_df_wildberries['title'].apply(process_title_r)

# 从“title"中提取服装种类信息(Извлеките информацию о типе одежды из раздела "название".)
def extract_name_r(title):
    names = ['пуховик', 'пальто', 'куртка', 'полушубок', 'бомбер', 'парка', 'шуба', 'дубленка', 'ветровка']
    for name in names:
        if name in title:
            return name
    return None


# 为数据集添加列“name"(Добавьте столбец “name" в набор данных)
csv_df_wildberries['name'] = csv_df_wildberries['title'].apply(extract_name_r)

Список “brand”

In [ ]:
# 删除列“brand"为空值的行(Удалите строки, столбец “brand" которых пуст)
csv_df_wildberries = csv_df_wildberries.dropna(subset=['brand'])

Список “gender”

In [ ]:
# 由于代表性别的“男"，“女"，没有固定出现的列，所以在每行的所有列中进行搜索(Поскольку нет фиксированных столбцов, представляющих пол “мужчина" и “женщина", выполните поиск по всем столбцам каждой строки.)
def extract_gender_r(dataset):
    genders = []
    # 遍历数据框的每一行(Пройдите по каждой строке фрейма данных)
    for index, row in dataset.iterrows():
        # 用于标记是否找到了性别信息(Используется для обозначения того, была ли найдена гендерная информация)
        gender_found = False
        # 遍历当前行的所有值(Обход всех значений текущей строки)
        for value in row.values:
            if isinstance(value, str):
                # 搜索性别信息(Поиск гендерной информации)
                if 'муж' in value and 'жен' in value:
                    genders.append('мужской и женский')
                    gender_found = True
                    break
                elif 'муж' in value:
                    genders.append('мужской')
                    gender_found = True
                    break
                elif 'жен' in value:
                    genders.append('женский')
                    gender_found = True
                    break
        if not gender_found:  # 如果整行都没有找到性别信息返回空值（Если информация о поле не найдена во всей строке, верните нулевое значение）
            genders.append(None)
    return genders

# 为数据集添加列“gender"（Добавьте столбец “gender" в набор данных）
csv_df_wildberries['gender'] = extract_gender_r(csv_df_wildberries)

Список “size”

In [ ]:
'''
1.尺码主要使用了俄罗斯标准和国际标准，为与中国数据作对比，统一转变为国际标准。
（1）.由于没有一个标准的写法，并且存在写了多个尺码的情况，所以取了可以搜索到的第一个尺码。
2.在俄罗斯在线购物网站中女性冬季外套数量远远大于男性外套数量，所以如果性别信息为空，默认使用女性尺码。

1.При расчете размера в основном используются российские стандарты и международные нормативы. Для сравнения с китайскими данными он был единообразно преобразован в международный стандарт.
（1）.Поскольку стандартного метода записи не существует, а бывают случаи, когда записывается несколько размеров, берется первый размер, который можно найти.
2.На российских сайтах интернет-магазинов количество женских зимних курток намного больше, чем мужских, поэтому, если информация о поле не указана, по умолчанию используется женский размер.
'''
# 从info_dict中提取尺码、颜色、材质和生产地信息（Извлеките информацию о размере, цвете, материале и месте производства из info_dict）
def extract_info_r(info_str, key):
    info_dict = literal_eval(info_str)
    return info_dict.get(key, None)

# 为数据集添加列“size"（Добавьте столбец “size" в набор данных）
csv_df_wildberries['size'] = csv_df_wildberries['info_dict'].apply(lambda x: extract_info_r(x, 'размер на модели'))


# 定义俄罗斯尺码表（Определите российскую таблицу размеров）
male_russian_size_chart = {'44': 'xs', '46': 's', '48': 'm', '50': 'l', '52': 'xl', '54': 'xl', '56': 'xxl', '58': 'xxl', '60': 'xxxl', '62': 'xxxxl'}
female_russian_size_chart = {'36': 'xxxs', '38': 'xxs', '40': 'xs', '42': 's', '44': 's', '46': 'm', '48': 'l', '50': 'l', '52': 'xl', '54': 'xxl',
                '56': 'xxxl', '58': 'xxxl'}

# 定义国际尺码表（Определите международную таблицу размеров）
male_international_size_chart = ['xs', 's', 'm', 'l', 'l/xl', 'xl', '2xl', '3xl', '3xl/4xl', '5xl']
female_international_size_chart = ['3xs', '2xs', 'xs', 's', 's/m', 'm', 'l', 'l/xl', 'xl', '2xl', '3xl', '4xl']

# 定义函数用于检查尺码是否在指定的尺码表中（Функция определения используется для проверки соответствия размера указанной таблице размеров）
def find_matching_size(sizes, size_chart):
    for size in sizes:
        # 如果尺码表是字典（Если таблица размеров является словарем）
        if isinstance(size_chart, dict):
            if size in size_chart:
                return size_chart[size]
        # 如果尺码表是列表（Если таблица размеров представляет собой список）
        elif isinstance(size_chart, list):
            if size in size_chart:
                return size
    return None

# 对列“size"进行搜索和替换（Найдите и замените столбец "size"）
csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace('-', '/', regex=False)
csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace('|', '/', regex=False)
# 替换括号及其中内容为空字符串（Замените скобки и содержимое пустой строкой）
csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace(r'\（.*?\）', '')
csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace(r'\rus', '')
csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace(r'\\eur', '')


# 遍历数据集中的每一行（Пройдите по каждой строке в наборе данных）
for index, row in csv_df_wildberries.iterrows():
    gender = row['gender']
    # 如果尺码不是 None，则按斜杠分割尺码字符串，否则设置为 [None]（Если размер не равен None, строка размера разделяется косой чертой, в противном случае устанавливается значение [None].）
    sizes = row['size'].split('/') if pd.notnull(row['size']) else [None]
    # 使用男性尺码表（Воспользуйтесь таблицей размеров для мужчин）
    if gender == 'мужской' or gender == 'мужской и женский':
        converted_size = find_matching_size(sizes, male_russian_size_chart)
        if converted_size is None:
            converted_size = find_matching_size(sizes, male_international_size_chart)
    # 检查女性尺码表（Ознакомьтесь с таблицей размеров для женщин）
    elif gender == 'женский':
        converted_size = find_matching_size(sizes, female_russian_size_chart)
        if converted_size is None:
            converted_size = find_matching_size(sizes, female_international_size_chart)
    # 处理空值情况（Обрабатывать ситуации с нулевым значением）
    else:
        # 检查女性尺码表（Ознакомьтесь с таблицей размеров для женщин）
        converted_size = find_matching_size(sizes, female_russian_size_chart)
        if converted_size is None:
            converted_size = find_matching_size(sizes, female_international_size_chart)

    # 更新数据集（Обновите набор данных）
    csv_df_wildberries.at[index, 'size'] = converted_size


# 定义替换字典（Определение словаря замены）
replacement_dict = {
    "3xs": "xxxs",
    "2xs": "xxs",
    "2xl": "xxl",
    "3xl": "xxxl",
    "4xl": "xxxxl"
}

# 将列“size"中的值进行替换（Замените значение в графе "size"）
csv_df_wildberries['size'] = csv_df_wildberries['size'].replace(replacement_dict)

# 对列“size"进行搜索和替换（Найдите и замените столбец "size"）
csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace(r'.*/', '')

<ipython-input-23-c6e4d8726397>:41: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace(r'\（.*?\）', '')
<ipython-input-23-c6e4d8726397>:43: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace(r'\rus', '')
<ipython-input-23-c6e4d8726397>:44: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace(r'\\eur', '')
<ipython-input-23-c6e4d8726397>:86: FutureWarning: The default value of regex will change from True to False in a future version.
  csv_df_wildberries['size'] = csv_df_wildberries['size'].str.replace(r'.*/', '')


Список “color”

In [ ]:
# 为数据集添加列“color"（Добавьте столбец “color" в набор данных）
csv_df_wildberries['color'] = csv_df_wildberries['info_dict'].apply(lambda x: extract_info_r(x, 'цвет'))

# 只保留主要颜色（Сохраняйте только основной цвет）
csv_df_wildberries['color'] = csv_df_wildberries['color'].str.split(';|,').str[0]
# 移除符号之后的内容（Содержимое после удаления символа）
csv_df_wildberries['color'] = csv_df_wildberries['color'].str.replace(r';.*|,.*', '', regex=True)
# 将列“color"中的“черный"替换为“чёрный"（Заменить "черный“ в графе "цвет“ на "цветной“.）
csv_df_wildberries['color'] = csv_df_wildberries['color'].str.replace('черный', 'чёрный', regex=False)

Список “material”

In [ ]:
'''
1.在俄罗斯在线购物网站中，人们对每种外套的Утеплитель都会关注，但在中国通常外套里外的材质是一样的，
 所以人们基本只在购买羽绒服时关注Утеплитель，甚至于其他种类的外套很少会有Утеплитель信息。
2.这里以相关信息更少的中国服装的信息为准，所以只保留Пуховик的Утеплитель信息。

1.На российских сайтах интернет-магазинов люди обращают внимание на утеплитель каждого пальто, но в Китае материал внутри и снаружи пальто обычно один и тот же.，
Поэтому люди в основном обращают внимание только на утеплитель при покупке пуховиков, и даже о других типах курток редко есть информация об утеплителе.
2.В данном случае преимущественную силу имеет информация о китайской одежде с менее значимой информацией, поэтому должна быть сохранена только вспомогательная информация о Puovik.
'''

# 为数据集添加列“material"（Добавьте столбец “material" в набор данных）
csv_df_wildberries['material'] = csv_df_wildberries['info_dict'].apply(lambda x: extract_info_r(x, 'состав'))

# 删除列“material”为空值的行（Удалите строки, столбец “material” которых пуст）
csv_df_wildberries = csv_df_wildberries.dropna(subset=['material'])

# 将列“material"中的每个值转换为字符串类型，然后再应用函数（Преобразуйте каждое значение в столбце “material" в строковый тип, а затем примените функцию）
csv_df_wildberries['material'] = csv_df_wildberries['material'].astype(str)
# 定义函数，用于去“%"，和数字（Определите функции для перехода к "%" и числам）
def clean_text(text):
    return re.sub(r'[%\d]+', '', text)
# 对列“material"应用函数（Примените функцию к столбцу "material"）
csv_df_wildberries['material'] = csv_df_wildberries['material'].apply(clean_text)


# 找到列“name"中数值为“Пуховик"的行（Найдите строку со значением “Пуховик" в столбце “name".）
down_jackets_data_r = csv_df_wildberries[csv_df_wildberries['name'] == 'Пуховик']

# 提取列“info_dict"中的项“Утеплитель"和的内容，并追加到“材质"项中（Извлеките содержимое пунктов “Помощник" и “Администратор" из столбца "info_dict" и добавьте их в пункт "material"）
for index, row in down_jackets_data_r.iterrows():
    info_dict = literal_eval(row['info_dict'])
    fill_material = info_dict.get('Утеплитель', '')
    new_material = row['material'] + '，' + fill_material
    csv_df_wildberries.at[index, 'material'] = new_material


# 将列“material"中的“;”替换为“,”（Замените “;" в графе “material” на “,”）
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace(';', ',', regex=False)
# 将列“material"中的“.”替换为“,”（Замените “." в графе “material” на “,”）
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('.', ',', regex=False)
# 替换括号及其中内容为空字符串(Замените скобки и содержимое пустой строкой)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace(r'\（.*?\）', '')
# 去除列“material"中的干扰内容(Удалите мешающий контент в графе “material".)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('твилpu', 'полиэстер', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('пэпуходержащий', 'полиэстер', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('верх:', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('основнойматериал:', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('подкладка:', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('наполнитель:', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('утеплитель:', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('тканьверха:', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('верх-', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('основнойматериал-', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('подкладка-', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('наполнитель-', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('утеплитель-', '', regex=False)
csv_df_wildberries['material'] = csv_df_wildberries['material'].str.replace('верхнийматериал-', '', regex=False)

# 使用正则表达式替换任意数量的空格字符为一个空格字符(Используйте регулярные выражения для замены любого количества пробелов одним символом)
csv_df_wildberries['material'] = csv_df_wildberries['material'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

<ipython-input-25-45be2d09e6ab>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_df_wildberries['material'] = csv_df_wildberries['material'].astype(str)
<ipython-input-25-45be2d09e6ab>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_df_wildberries['material'] = csv_df_wildberries['material'].apply(clean_text)
<ipython-input-25-45be2d09e6ab>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

 Список“origin”

In [ ]:
# 为数据集添加列“origin"(Добавьте столбец “origin" в набор данных)
csv_df_wildberries['origin'] = csv_df_wildberries['info_dict'].apply(lambda x: extract_info_r(x, 'страна производства'))

 Список “price”,“count”

In [ ]:
# 将列“price”转换为整数型(Преобразуйте столбец “price” в целочисленный тип)
csv_df_wildberries['price'] = csv_df_wildberries['price'].astype(int)


# 将列“count”转换为浮点数型，将无法转换的非数值设为 NaN（Преобразуйте столбец “count” в число с плавающей запятой и задайте нечисловое значение, которое не может быть преобразовано в NaN）
csv_df_wildberries['count'] = pd.to_numeric(csv_df_wildberries['count'], errors='coerce')
# 删除列“count”为空值的行（Удалить строки, столбец “count” которых пуст）
csv_df_wildberries = csv_df_wildberries.dropna(subset=['count'])
# 将列“count”转换为整数型（Преобразуйте столбец “count” в целочисленный тип）
csv_df_wildberries['count'] = csv_df_wildberries['count'].astype(int)

<ipython-input-27-0a9eee47d639>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_df_wildberries['count'] = csv_df_wildberries['count'].astype(int)


Список “rating”

In [ ]:
'''
1.为方便计算将列“rating"的值转变为小于1，大于0的浮点数。
2.由于两个网站都没有商品销售数量数据，而有商品评论数量数据（列“count"）和商品评分数据（列“rating"），所以对这两种数据进行处理合并以代表好评数值。
3.将商品评论数（列“count"）和商品评分数据（列“rating"）相乘，得到基于商品评论数（列“count"）和商品评分数据（列“rating"）的好评数值，并创建新列“sales_index"。

1.Чтобы облегчить расчет, значение столбца “rating" преобразуется в число с плавающей запятой, которое меньше 1 и больше 0.
2.Поскольку ни на одном из веб-сайтов нет данных о количестве продаж продукта, кроме данных о количестве отзывов о продукте (столбец “count") и данных о рейтинге продукта (столбец “rating"), эти две данные обрабатываются и объединяются для представления значения оценки.
3.Умножьте количество отзывов о продукте (столбец “количество") и данные о рейтинге продукта (столбец “рейтинг"), чтобы получить значение похвалы, основанное на количестве отзывов о продукте (столбец “count") и данных о рейтинге продукта (столбец “rating"), и создайте новый столбец “sales_index".
'''

# 将列“rating"变为浮点数（Измените столбец “rating" на число с плавающей запятой）
csv_df_wildberries['rating'] = csv_df_wildberries['rating'].apply(lambda x: float(x) * 20 / 100 if isinstance(x, str) else x)

# 将列“count"和列“rating"相乘得到销售综合指标（#Умножьте столбец “count" на столбец “rating", чтобы получить общий показатель продаж.）
sales_index = csv_df_wildberries['count'] * csv_df_wildberries['rating']
# 创建新列“sales_index”（Создайте новый столбец "sales_index”）
csv_df_wildberries['sales_index'] = sales_index

<ipython-input-28-bbaae16f65fb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_df_wildberries['rating'] = csv_df_wildberries['rating'].apply(lambda x: float(x) * 20 / 100 if isinstance(x, str) else x)
<ipython-input-28-bbaae16f65fb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_df_wildberries['sales_index'] = sales_index


Список “sales_index”

In [ ]:
# 定义区间和对应的标签（Определите интервал и соответствующую метку）
bins = [-float('inf'), 100, 200, 500, 1000, 2000, 3000, 4000, 5000, float('inf')]
labels = ['0-100', '100-200', '200-500', '500-1000', '1000-2000', '2000-3000', '3000-4000', '4000-5000', '5000+']

# 使用cut函数将数值划分为区间并添加区间标签（Используйте функцию "Вырезать", чтобы разделить значение на интервалы и добавить метки интервалов）
csv_df_wildberries['sales_index'] = pd.cut(csv_df_wildberries['sales_index'], bins=bins, labels=labels, right=False)

<ipython-input-29-316bc4a63fc8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_df_wildberries['sales_index'] = pd.cut(csv_df_wildberries['sales_index'], bins=bins, labels=labels, right=False)


删除特征列和调整特征列顺序及保存修改后的数据集（Удалите столбцы объектов, настройте порядок столбцов объектов и сохраните измененный набор данных.）

In [ ]:
# 删除“info_dict"，“count"，“rating"列（Удалите столбец "info_dict“, “count"， "rating"）
csv_df_wildberries.drop(columns = [ 'info_dict', 'count', 'rating'], inplace = True)

# 调整列的顺序（Измените порядок следования столбцов）
new_column_order = ['title', 'name', 'brand', 'gender', 'size', 'color', 'material', 'origin', 'price', 'sales_index']
wildberries_data_1 = csv_df_wildberries[new_column_order]

# 处理后的数据存储到新的CSV文件中（Обработанные данные сохраняются в новом CSV-файле）
output_csv_wildberries = '/content/wildberries冬季外套数据（处理后1.0）.csv'
fieldnames = ['title', 'name', 'brand', 'gender', 'size', 'color', 'material', 'origin', 'price', 'sales_index']

with open(output_csv_wildberries, 'w', encoding = 'utf-8', newline= '') as file:
    writer_wildberries = csv.DictWriter(file, fieldnames = fieldnames)

    # 写入表头（Написать заголовок）
    writer_wildberries.writeheader()

    # 写入处理后的数据（Запишите обработанные данные）
    writer_wildberries.writerows(wildberries_data_1.to_dict(orient = 'records'))

<ipython-input-30-b200ef20711b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_df_wildberries.drop(columns = [ 'info_dict', 'rating', 'count'], inplace = True)


In [ ]:
# 读取wildberries数据集（Ознакомьтесь с набором данных wildberries）
csv_wildberries_data_1 = "/content/wildberries冬季外套数据（处理后1.0）.csv"
csv_data_wildberries_data_1 = pd.read_csv(csv_wildberries_data_1, low_memory = False)
csv_df_wildberries_data_1 = pd.DataFrame(csv_data_wildberries_data_1)
csv_df_wildberries_data_1

,title,name,brand,gender,size,color,material,origin,price,sales_index
0,пальто демисезонное экомех оверсайз тедди,пальто,baimuni,NaN,s,белый,"эко кожа , эко шерсть",китай,7087,500-1000
1,куртка весна с капюшоном,куртка,tanir,NaN,s,чёрный,полиэстер,россия,4597,100-200
2,косуха куртка серая оверсайз,куртка,moraru,женский,xs,бело-серый,экокожа,россия,2316,5000+
3,пальто демисезонное шерстяное оверсайз,пальто,overcoat,женский,NaN,чёрный,"шерсть , вискоза",россия,12203,500-1000
4,пальто женское оверсайз веснаосень демисезонное,пальто,bregato,женский,s,чёрный,"вискоза, шерсть",россия,4722,200-500
...,...,...,...,...,...,...,...,...,...,...
2839,пальто мужское демисезонное оверсайз,пальто,quatro,мужской,s,чёрный,"вискоза , полиэстер , шерсть",россия,6419,0-100
2840,куртка бомбер демисезонная,куртка,tommy jeans,NaN,NaN,темно-синий,полиамид,китай,11275,0-100
2841,пальто весеннее длинное с капюшоном оверсайз,пальто,monomir discount,женский,s,молочный,полиэстер,китай,11494,3000-4000
2842,кардиган длинный оверсайз вязаный с пуговицами,NaN,monpele,женский,NaN,темная лаванда,шерсть акрил,NaN,1861,5000+
